In [3]:
import numpy as np
import copy
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import math
%matplotlib inline

# Qusetion
$$ \frac{\partial u}{\partial t}-\frac{\partial^{2} u}{\partial x^{2}} = 0$$

$$0< x<2,0<t \\ u(0,t)=u(2,t)=0,0<t  \\ u(x,0) = sin(\frac{\pi}{2}x)$$


## exact soluation

In [16]:
def exact_soluation(x_range, t_range):
    ans = []
    
    ans.append(math.exp(-math.pi*t/4)*math.sin(math.pi*x/2)) 
    return ans

In [12]:
x = np.linspace(0, 2, num=10)
tim = np.linspace(0,100,num = 100)

In [13]:
exact_soluation(x,tim)

TypeError: only size-1 arrays can be converted to Python scalars

In [14]:
x

array([0.        , 0.22222222, 0.44444444, 0.66666667, 0.88888889,
       1.11111111, 1.33333333, 1.55555556, 1.77777778, 2.        ])

In [15]:
tim

array([  0.        ,   1.01010101,   2.02020202,   3.03030303,
         4.04040404,   5.05050505,   6.06060606,   7.07070707,
         8.08080808,   9.09090909,  10.1010101 ,  11.11111111,
        12.12121212,  13.13131313,  14.14141414,  15.15151515,
        16.16161616,  17.17171717,  18.18181818,  19.19191919,
        20.2020202 ,  21.21212121,  22.22222222,  23.23232323,
        24.24242424,  25.25252525,  26.26262626,  27.27272727,
        28.28282828,  29.29292929,  30.3030303 ,  31.31313131,
        32.32323232,  33.33333333,  34.34343434,  35.35353535,
        36.36363636,  37.37373737,  38.38383838,  39.39393939,
        40.4040404 ,  41.41414141,  42.42424242,  43.43434343,
        44.44444444,  45.45454545,  46.46464646,  47.47474747,
        48.48484848,  49.49494949,  50.50505051,  51.51515152,
        52.52525253,  53.53535354,  54.54545455,  55.55555556,
        56.56565657,  57.57575758,  58.58585859,  59.5959596 ,
        60.60606061,  61.61616162,  62.62626263,  63.63